In [26]:
def coupang_crawling(keyword, item_count_start, item_count_end):
    import requests
    from scrapy.http import TextResponse
    import json
    search_url = "http://www.coupang.com/np/search?component=&q={}".format(keyword)
    rep = requests.get(search_url)
    response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

    # 상품마다의 링크를 불러오기에 앞서 아이템 Top 10 리스트 불러오기
    products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][item_count_start:item_count_end]

    df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

    review_urls = {}

    for product in products:

        # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
        try:
    #         review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
            review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
        except Exception as e:
            print('No reviews')
            
        def make_url(product, review_count):
            page_count = (review_count//100)+1
            review_urls = []
            for page in range(1, page_count+1):
                review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
                # size는 100이 최대로 설정되어 있음
            return review_urls


        # Top 10 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
        review_urls[product] = make_url(product, review_count)
        review_url = make_url(product, review_count) # 리스트로 반환됨
    #     print(review_url)

        for url in review_url:

            review_url = url
            review_rep = requests.get(review_url)
            review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

            # Top 10 상품마다의 상품평 개수
            review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

            # Top 10 상품마다 한 page의 상품평 개수
            review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

            if review_url_cnt < 100:          
                for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                    item_nbr = product

                    date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                    name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                    item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                    if len(item_option_ls) == 1:
                        item = item_option_ls[0]

                    else:    
                        item = item_option_ls[0]
                        option = item_option_ls[1]

                    if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                        re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                        review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                    else:
                        re_title = ['.']
                        review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                    rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                    df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

            else:
                for i in range(1, 101):
                    item_nbr = product

                    date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                    name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                    item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                    if len(item_option_ls) == 1:
                        item = item_option_ls[0]

                    else:    
                        item = item_option_ls[0]
                        option = item_option_ls[1]

                    if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                        re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                        review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                    else:
                        re_title = ['.']
                        review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                    rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                    df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                    
    def re_title_filter(a):
        for i in range(len(a[0].split('\n'))):
            if len(a[0].split('\n')[i].strip()) != 0:
                return a[0].split('\n')[i].strip()
            else:
                return '.'

    def review_filter(a):
        s = ''
        for i in range(len(a)):
            for ii in range(len(a[i].split('\n'))):
                if a[i].split('\n')[ii].strip() != 0:
                    s += a[i].split('\n')[ii].strip() + ' '
        return s

    def kkma_pos(a):
        from konlpy.tag import Kkma
        kkma = Kkma()
        return kkma.pos(a)

    def kkma_pos_filter(a):
        """
        df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
        """
        ls = []
        for i in range(len(a)):
            #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
            if a[i][1]=='NNG' or a[i][1]=='NNP' or a[i][1]=='VA' or a[i][1]=='UN' or a[i][1]=='XR' or a[i][1]=='MAG':
                ls.append(a[i][0])
        return ls

    def rating_filter(a):
        if a == '3' or a == '4' or a == '5':
            return 1
            # 긍정
        else:
            return 0
            # 부정
    
    df['re_title_filtered'] = df['re_title'].apply(re_title_filter)
    df['review_filtered'] = df['review'].apply(review_filter)
    df['full_review'] = df['re_title_filtered']+df['review_filtered']
    df['pos'] = df['full_review'].apply(kkma_pos)
    df['pos_filtered'] = df['pos'].apply(kkma_pos_filter)
    df['rating_filtered'] = df['rating'].apply(rating_filter)
    
    df_fin = df[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
    
    return df_fin

In [10]:
df_train = coupang_crawling('샘키즈', 0, 10)
df_train.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
758,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,B형 (몸통)크림화이트/(박스)하프코랄(B),2018.06.01,정*은,넘 좋아요 그동안 쇼파위에 가득히 올려져있던 장난감들 여자아이라 작고 큰 많은 ...,"[(넘, MAG), (좋, VA), (아요, EFN), (그, MDT), (동안, ...","[넘, 좋, 동안, 쇼, 파, 위, 가득히, 장난감, 여자, 작, 크, 많, 인형,..."
759,3,0,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,C형 (몸통)메이플/(박스)코랄그레이(C),2018.06.16,이*진,. 아직 상품 받지 않았는데 배송 완료는 뭔가요? 상품 받지 않은 중에 사이...,"[(., SF), (아직, MAG), (상품, NNG), (받, VV), (지, E...","[아직, 상품, 배송, 완료, 가요, 상품, 중, 사이즈, 교환, 코, 자, 상담,..."
760,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,B형 (몸통)크림화이트/(박스)하프코랄(B),2017.11.05,오*종,좋아요.. ^^ 국민 서랍장이란 말이 맞네요. 공간 사용이 좋아지네요. 빨리 사면...,"[(좋, VA), (아요, EFN), (.., SW), (^^, EMO), (국민,...","[좋, 국민, 서랍, 말, 공간, 사용, 빨리, 이득]"
761,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,A형 (몸통)크림화이트/(박스)코랄그레이(A),2018.05.26,정*채,두번째 또구입 이번이 두번째 구매에요 첫번째는 장난감 정리 두번째는 옷정리~...,"[(두, MDN), (번째, NNB), (또, MAG), (구입, NNG), (이번...","[또, 구입, 이번, 구매, 장난감, 정리, 옷, 정리, 좁, 형도, 보기, 좋, ..."
762,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,B형 (몸통)크림화이트/(박스)하프코랄(B),2017.12.24,최*경,.,"[(., SF)]",[]


In [ ]:
df_train = coupang_crawling('샘키즈', 0, 10)
df_train.tail()

In [ ]:
# df_train로 만든 단어장
# train 데이터와 test 데이터의 단어장은 동일해야한다

In [4]:
def Vectorizer(df_train, pos_filtered_col, df_input):
    """
    df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    train_corpus = []
    for i in range(len(df_train[pos_filtered_col])):
        for ii in range(len(df_train[pos_filtered_col][i])):
            train_corpus.append(df_train[pos_filtered_col][i][ii])
            
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b") # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(train_corpus)

    vect_ls = []
    for i in range(len(df_input)):
        vect_ls.append((vect.transform(df_input[pos_filtered_col][i]).toarray().sum(axis=0).tolist()))
    
    df_vec = pd.DataFrame(vect_ls)
    return df_vec

In [18]:
X_train = Vectorizer(df_train, 'pos_filtered', df_train)
y_train = df_train['rating_filtered']

In [25]:
df_train.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
758,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,B형 (몸통)크림화이트/(박스)하프코랄(B),2018.06.01,정*은,넘 좋아요 그동안 쇼파위에 가득히 올려져있던 장난감들 여자아이라 작고 큰 많은 ...,"[(넘, MAG), (좋, VA), (아요, EFN), (그, MDT), (동안, ...","[넘, 좋, 동안, 쇼, 파, 위, 가득히, 장난감, 여자, 작, 크, 많, 인형,..."
759,3,0,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,C형 (몸통)메이플/(박스)코랄그레이(C),2018.06.16,이*진,. 아직 상품 받지 않았는데 배송 완료는 뭔가요? 상품 받지 않은 중에 사이...,"[(., SF), (아직, MAG), (상품, NNG), (받, VV), (지, E...","[아직, 상품, 배송, 완료, 가요, 상품, 중, 사이즈, 교환, 코, 자, 상담,..."
760,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,B형 (몸통)크림화이트/(박스)하프코랄(B),2017.11.05,오*종,좋아요.. ^^ 국민 서랍장이란 말이 맞네요. 공간 사용이 좋아지네요. 빨리 사면...,"[(좋, VA), (아요, EFN), (.., SW), (^^, EMO), (국민,...","[좋, 국민, 서랍, 말, 공간, 사용, 빨리, 이득]"
761,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,A형 (몸통)크림화이트/(박스)코랄그레이(A),2018.05.26,정*채,두번째 또구입 이번이 두번째 구매에요 첫번째는 장난감 정리 두번째는 옷정리~...,"[(두, MDN), (번째, NNB), (또, MAG), (구입, NNG), (이번...","[또, 구입, 이번, 구매, 장난감, 정리, 옷, 정리, 좁, 형도, 보기, 좋, ..."
762,5,1,32052473,한샘 샘키즈 수납장 1005 블라썸 코랄_좁은형,B형 (몸통)크림화이트/(박스)하프코랄(B),2017.12.24,최*경,.,"[(., SF)]",[]


In [24]:
X_train.tail()

,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
758,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
759,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
760,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
761,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
762,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.96      0.41      0.57        64
          1       0.95      1.00      0.97       699

avg / total       0.95      0.95      0.94       763



In [29]:
df_test = coupang_crawling('샘키즈', 0, 36)
df_test.tail()

No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
866,5,1,32029717,한샘 샘키즈 수납장 670 블라썸 코랄_좁은형,B형 (몸통)크림화이트/(박스)하프코랄(B),2018.01.20,변*정,.,"[(., SF)]",[]
867,5,1,32029717,한샘 샘키즈 수납장 670 블라썸 코랄_좁은형,C형 (몸통)메이플/(박스)코랄그레이(C),2017.12.12,서*영,.,"[(., SF)]",[]
868,5,1,120477034,샘키즈 스칸디 에디션 3단수납책장 870 B타입 (2종/택,B: 메이플(B),2018.05.29,김*정,맘에 들어요~~~,"[(맘, NNG), (에, JKM), (듣, VV), (어요, EFN), (~~~,...",[맘]
869,5,1,120477034,샘키즈 스칸디 에디션 3단수납책장 870 B타입 (2종/택,A: 화이트(A),2018.04.18,김*애,.,"[(., SF)]",[]
870,5,1,32101157,[한샘] 샘키즈 2단책장 870 (2종/택1),B형:메이플(B),2017.12.29,하*수,.,"[(., SF)]",[]


In [30]:
X_test = Vectorizer(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [31]:
X_test.tail()

,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
866,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
869,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
870,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.97      0.39      0.56        74
          1       0.95      1.00      0.97       797

avg / total       0.95      0.95      0.94       871



In [33]:
df_test[df_test['rating_filtered'] == 0]

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
0,2,0,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),A: 화이트(A),2018.04.16,윤*영,. 아기방물건 정리하려구 구매했어요. 물건은 지정한배송일에왔구요 근데 새가구인...,"[(., SF), (아기방, NNG), (물건, NNG), (정리, NNG), (하...","[아기방, 물건, 정리, 구매, 물건, 지정, 배송, 일, 근데, 가구, 찍힘, 설..."
3,2,0,32033206,샘키즈 스칸디 에디션 수납장 1005 (2종/택1),A: 화이트(A),2018.02.22,강*선,. 배송도 늦고 불친절하고 방향도 틀려서 다시하고 진짜 써비스 맘에 안들었어요~ ...,"[(., SF), (배송, NNG), (도, JX), (늦, VA), (고, ECE...","[배송, 늦, 불친절, 방향, 다시, 진짜, 써비스, 맘, 안, 박스, 기, 고, 그냥]"
24,1,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2018.02.24,이*교,기사님 서비스 최악 상품은 둘째치고 배송기사분 정말 최악이네요 집에 와이프가 애...,"[(기사님, NNP), (서비스, NNG), (최악, NNG), (상품, NNG),...","[기사님, 서비스, 최악, 상품, 배송, 기사분, 정말, 최악, 집, 와이프, 애기..."
39,3,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,C형:(몸통)크림화이트/(박스)코랄민트(C),2018.01.12,김*정,쓸만합니다 좋네여. 근데 조립이 아쉽네요 아무래도 새건데 일케 들뜨면 구매자...,"[(쓰, VV), (ㄹ, ETD), (만하, VXA), (ㅂ니다, EFN), (좋,...","[좋, 근데, 조립, 아쉽, 아무래도, 일, 구매자, 입장, 기분, 안, 좋, ㅠ,..."
45,3,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2017.10.24,김*희,상품은좋은데배송이.. 상품은좋아요 근데 배송기사님 배정이 당일새벽에 된다며 미...,"[(상품, NNG), (은, JX), (좋, VA), (은데, ECD), (배송, ...","[상품, 좋, 배송, 상품, 좋, 근데, 배송, 기사님, 배정, 당일, 새벽, 미리..."
92,3,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,D형:(몸통)메이플/(박스)민트그레이(D),2018.08.11,강*혜,. 물건은 깔끔하나 생각보다 크다고했는데 그닥 큰줄모르겠고 조립이 엉망입니다 기사...,"[(., SF), (물건, NNG), (은, JX), (깔끔, XR), (하, XS...","[물건, 깔끔, 생각, 크, 그닥, 크, 조립, 엉망, 기사분, 가신, 후, 틀, ..."
93,1,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,C형:(몸통)크림화이트/(박스)코랄민트(C),2018.01.09,이*경,. 이 상품 짜증나요.. 프라스틱 서랍 트리오로 닦으라해서 닦는데 겹쳐서 와서 ...,"[(., SF), (이, JKS), (상품, NNG), (짜증나, VV), (아요,...","[상품, 프, 스틱, 서랍, 트리오, 잘, 쌍둥이, 반품, 처리, 박스, 어찌나, 더럽]"
94,2,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2018.02.09,도래도래,. 글쎄요 서랍장이 지저분해요 기사님도 별루였구요 음료수준비룰 못해서 죄송했지만 ...,"[(., SF), (글쎄, IC), (요, NNG), (서랍, NNG), (장이, ...","[요, 서랍, 지저분, 기사님, 별루, 음료수, 준비, 룰, 못하, 죄송, 다행, ..."
140,3,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2018.04.09,임*연,. 주중에 낫동안 집에 사람이 없다보니 토요일 배송기다리는데 2주걸렸습니다,"[(., SF), (주중, NNG), (에, JKM), (낫, NNG), (동안, ...","[주중, 낫, 동안, 집, 사람, 없, 토요일, 배송]"
143,2,0,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2018.07.18,강*선,. 걍 쏘쏘 생각보다 작음 설치하는 아저씨들도 그냥저냥 그닥 친절하지 않음,"[(., SF), (걍, MAG), (쏘쏘, UN), (생각, NNG), (보다, ...","[걍, 쏘쏘, 생각, 음, 설치, 아저씨, 그냥저냥, 그닥, 친절]"


In [2]:
df_train = coupang_crawling('조명', 0, 5)
df_train.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
10583,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,옐로우,2016.01.20,김*순,.,"[(., SF)]",[]
10584,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,실버,2016.01.19,이*선,.,"[(., SF)]",[]
10585,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.17,김*훈,.,"[(., SF)]",[]
10586,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2016.01.05,김*연,.,"[(., SF)]",[]
10587,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.02,김*숙,.,"[(., SF)]",[]


In [5]:
X_train = Vectorizer(df_train, 'pos_filtered', df_train)
y_train = df_train['rating_filtered']

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.87      0.34      0.49      1262
          1       0.92      0.99      0.95      9326

avg / total       0.91      0.92      0.90     10588



In [17]:
df_test = coupang_crawling('조명', 6, 10)
df_test.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
6948,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,옐로우,2016.01.20,김*순,.,"[(., SF)]",[]
6949,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,실버,2016.01.19,이*선,.,"[(., SF)]",[]
6950,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.17,김*훈,.,"[(., SF)]",[]
6951,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2016.01.05,김*연,.,"[(., SF)]",[]
6952,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.02,김*숙,.,"[(., SF)]",[]


In [18]:
X_test = Vectorizer(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [19]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.86      0.27      0.42       792
          1       0.91      0.99      0.95      6161

avg / total       0.91      0.91      0.89      6953



In [21]:
idx = []
for i in range(len(df_test)):
    if len(df_test['pos_filtered'][i]) == 0:
        idx.append(i)

In [22]:
df_test_ = df_test.drop(index=idx).reset_index()

In [23]:
X_test_ = Vectorizer(df_train, 'pos_filtered', df_test_)
y_test_ = df_test_['rating_filtered']

In [24]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred_ = model.predict(X_test_)
print(classification_report(y_test_, y_test_pred_))

             precision    recall  f1-score   support

          0       0.86      0.52      0.64       421
          1       0.94      0.99      0.96      3296

avg / total       0.93      0.94      0.93      3717



In [25]:
df_test_[df_test_['rating_filtered']==0]

,index,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
14,14,2,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2017.10.24,정*빈,. 자꾸 넘어집니다 그리고 플라스틱이아 그런지 다른곳에서 산 쇠로 만든 플라밍...,"[(., SF), (자꾸, MAG), (넘어지, VV), (ㅂ니다, EFN), (그...","[자꾸, 플라스틱, 그렇, 곳, 쇠, 플라밍고, 차이, 마니, 다, 건전지, 사용,..."
15,15,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.01.10,진*,". 예뻐서 구매했으나, 바닥이 비스듬히 되어서 서있기가 위태롭고 자꾸 넘어집니다....","[(., SF), (예쁘, VA), (어서, ECD), (구매, NNG), (하, ...","[예쁘, 구매, 바닥, 비스듬히, 위태, 자꾸, 프, 스틱, 재질, 고급, 전혀, ..."
27,27,3,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2017.11.01,신*연,. 무드등인만큼 분위기내는데는 일품! 하지만 퀄리티는 싼가격만큼 별로네요 ㅠㅠ 근...,"[(., SF), (무드, NNG), (등인, NNG), (만큼, NNG), (분위...","[무드, 등인, 만큼, 분위기, 일품, 퀄리티, 가격, 별로, 요, 그, 성비, 좋..."
32,32,3,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.08.17,음오아에,. 상품은 마음에 드는데 다른 후기들과 다르게 저는 누가 밟았나 느껴질정도의 저...,"[(., SF), (상품, NNG), (은, JX), (마음, NNG), (에, J...","[상품, 마음, 다르, 후기, 누, 정도, 허름, 상자, 포장, 반품, 상품, 기분..."
40,40,3,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.01.16,이*혜,. 선인장이 앞으로 좀 쏠려요 ㅠㅠ 앞쪽으로 넘어질 것 같은 느낌 ㅜㅜㅜㅜ 교환 ...,"[(., SF), (선인, NNG), (장이, XSN), (앞, NNG), (으로,...","[선인, 앞, 좀, 앞쪽, 같, 느낌, 교환, 귀찮, 그냥]"
42,42,3,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.01.08,서*원,. 아이방에 놓아주니 이쁩니다 단 받침이 더 안정적이었으면 하는 아쉬움이 있어요...,"[(., SF), (아이, NNG), (방, NNG), (에, JKM), (놓아주,...","[아이, 방, 받침, 더, 안정적, 아쉬움]"
49,49,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.02.20,김*주,. 아니 왜 불이 안들어와요? 건전지 바꿔끼워도 안되요,"[(., SF), (아, VV), (니, ECD), (왜, MAG), (불이, NN...","[왜, 불이, 안, 건전지, 안]"
67,67,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.08.14,김*아,. 반품합니다. 무드등인데 건전지 넣는부분이 왜이러죠? 왠만하면 쓸려고했느데 ...,"[(., SF), (반품, NNG), (하, XSV), (ㅂ니다, EFN), (.,...","[반품, 무드, 건전지, 부분, 왜, 했느, 데, 불, 안, 반품, 가능, 빨리, ..."
79,79,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.02.27,이*진,. 이뻐서 구매했더니.. 뜯어보니 중고가 왔나했어요. 싸구려 플라스틱 재질인건 감...,"[(., SF), (이쁘, VV), (어서, ECD), (구매, NNG), (하, ...","[구매, 중고, 싸구려, 플라스틱, 재질, 인건, 감안, 스크래치, 작렬, 뒤, 건..."
96,96,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.07.28,박*해,. 스크레치에 건전지 끼우는 뒷면 부서져서 왔어요 잘좀 싸서 보내주시지 배달이 엉...,"[(., SF), (스크레치, UN), (에, JKM), (건전지, NNG), (끼...","[스크레치, 건전지, 끼우는, 뒷면, 잘, 좀, 배달, 엉망, 그렇, 품, 자체, ..."


In [ ]:
# 현재까지의 모든 데이터로 모델링 후
# 기간지정 데이터를 test 데이터로 만들어서 예측
# 상품평마다 긍정 / 부정 예측 후 총 데이터에서 긍정 / 부정 데이터 비중 출력
# 별점 상관없이 상품명 키워드만 보고 긍정 / 부정 예측하는 것이 목표 -> 테스트 데이터 수동으로 라벨링 한 후 다시 report 출력
# 긍정 데이터 주요 키워드 출력, 부정 데이터 주요 키워드 출력
# * 긍정, 부정 데이터 비대칭 해결해야함
# 부정적이지 않은 단어들의 filtering(부분적으로 부정 데이터에만 stop word 적용?)
# 새로운 단어 -> missing value X, 중요한 단어일 수도 있는데 그냥 through된다는 것
# 품사필터링 + stop word 설정으로 모델 업데이트 필요

# countvector 파이프라인 X(함수로 만들어서 적용하면 됨), tf-idf(성능보고 선택적으로)
# kkma 사용자 사전 추가(자바 빌드)

In [ ]:
def Vectorizer(df_train, pos_filtered_col, df_input):
    """
    df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    train_corpus = []
    for i in range(len(df_train[pos_filtered_col])):
        for ii in range(len(df_train[pos_filtered_col][i])):
            train_corpus.append(df_train[pos_filtered_col][i][ii])
            
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b") # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(train_corpus)

    vect_ls = []
    for i in range(len(df_input)):
        vect_ls.append((vect.transform(df_input[pos_filtered_col][i]).toarray().sum(axis=0).tolist()))
    
    df_vec = pd.DataFrame(vect_ls)
    return df_vec

In [ ]:
휴리스틱 튜닝
가짜 데이터 추가
stop word도 수동으로
나이브베이즈 parameter - sample weight '으나'

In [40]:
df_test_['pos'].iloc[15]

[('.', 'SF'),
 ('예쁘', 'VA'),
 ('어서', 'ECD'),
 ('구매', 'NNG'),
 ('하', 'XSV'),
 ('었', 'EPT'),
 ('으나', 'ECE'),
 (',', 'SP'),
 ('바닥', 'NNG'),
 ('이', 'JKS'),
 ('비스듬히', 'MAG'),
 ('되', 'VV'),
 ('어서', 'ECD'),
 ('스', 'VV'),
 ('어', 'ECS'),
 ('있', 'VXV'),
 ('기', 'ETN'),
 ('가', 'JKS'),
 ('위태', 'XR'),
 ('롭', 'XSA'),
 ('고', 'ECE'),
 ('자꾸', 'MAG'),
 ('넘어지', 'VV'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('프', 'UN'),
 ('이', 'VCP'),
 ('라', 'EFN'),
 ('스틱', 'NNG'),
 ('재질', 'NNG'),
 ('이', 'VCP'),
 ('라', 'ECD'),
 ('고급', 'NNG'),
 ('스럽', 'XSA'),
 ('ㅁ', 'ETN'),
 ('과', 'JKM'),
 ('는', 'JX'),
 ('전혀', 'MAG'),
 ('거리', 'NNG'),
 ('가', 'JKS'),
 ('멀', 'VV'),
 ('며', 'ECE'),
 (',', 'SP'),
 ('조잡', 'XR'),
 ('하', 'XSA'),
 ('ㄴ', 'ETD'),
 ('장난감', 'NNG'),
 ('수준', 'NNG'),
 ('의', 'JKG'),
 ('느낌', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('가격', 'NNG'),
 ('적', 'XSN'),
 ('으로', 'JKM'),
 ('고급', 'NNG'),
 ('적', 'XSN'),
 ('이', 'VCP'),
 ('ㄴ', 'ETD'),
 ('느낌', 'NNG'),
 ('을', 'JKO'),
 ('바라', 'VV'),
 ('ㄴ', 'ETD'),
 ('것', 'NNB'),
 ('은', 

In [39]:
df_test_[df_test_['rating_filtered']==0]

,index,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
14,14,2,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2017.10.24,정*빈,. 자꾸 넘어집니다 그리고 플라스틱이아 그런지 다른곳에서 산 쇠로 만든 플라밍...,"[(., SF), (자꾸, MAG), (넘어지, VV), (ㅂ니다, EFN), (그...","[자꾸, 플라스틱, 그렇, 곳, 쇠, 플라밍고, 차이, 마니, 다, 건전지, 사용,..."
15,15,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.01.10,진*,". 예뻐서 구매했으나, 바닥이 비스듬히 되어서 서있기가 위태롭고 자꾸 넘어집니다....","[(., SF), (예쁘, VA), (어서, ECD), (구매, NNG), (하, ...","[예쁘, 구매, 바닥, 비스듬히, 위태, 자꾸, 프, 스틱, 재질, 고급, 전혀, ..."
49,49,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.02.20,김*주,. 아니 왜 불이 안들어와요? 건전지 바꿔끼워도 안되요,"[(., SF), (아, VV), (니, ECD), (왜, MAG), (불이, NN...","[왜, 불이, 안, 건전지, 안]"
67,67,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.08.14,김*아,. 반품합니다. 무드등인데 건전지 넣는부분이 왜이러죠? 왠만하면 쓸려고했느데 ...,"[(., SF), (반품, NNG), (하, XSV), (ㅂ니다, EFN), (.,...","[반품, 무드, 건전지, 부분, 왜, 했느, 데, 불, 안, 반품, 가능, 빨리, ..."
79,79,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.02.27,이*진,. 이뻐서 구매했더니.. 뜯어보니 중고가 왔나했어요. 싸구려 플라스틱 재질인건 감...,"[(., SF), (이쁘, VV), (어서, ECD), (구매, NNG), (하, ...","[구매, 중고, 싸구려, 플라스틱, 재질, 인건, 감안, 스크래치, 작렬, 뒤, 건..."
96,96,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.07.28,박*해,. 스크레치에 건전지 끼우는 뒷면 부서져서 왔어요 잘좀 싸서 보내주시지 배달이 엉...,"[(., SF), (스크레치, UN), (에, JKM), (건전지, NNG), (끼...","[스크레치, 건전지, 끼우는, 뒷면, 잘, 좀, 배달, 엉망, 그렇, 품, 자체, ..."
117,117,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.08.14,민*정,. 무드등 선인장 두개를 구매하였으나 다른걸로 와서 시간이없어 그냥사용했습니다. ...,"[(., SF), (무드, NNG), (등, NNB), (선, NNG), (인장, ...","[무드, 선, 인장, 두개, 구매, 다르, 시간, 없, 그냥, 구매, 상품, 정확히..."
123,123,2,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.04.22,김*진,. 솔직히 좋은줄 모르겠어요.배터리 부분 접촉이 안좋아 불이 들어왔다 안들어왔다 ...,"[(., SF), (솔직히, MAG), (좋, VA), (은, ETD), (줄, N...","[솔직히, 좋, 배터리, 부분, 접촉, 안, 좋, 불이, 안, 세우, 기도, 불편,..."
141,141,2,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.08.18,이*숙,. 선인장을 구매했는데 박스에는 선인장으로 되어 있고 내용물은 홍학이 왔네요 아쉽...,"[(., SF), (선인장, NNG), (을, JKO), (구매, NNG), (하,...","[선인장, 구매, 박스, 선인장, 내용물, 홍학, 그냥]"
166,166,1,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.05.13,김*미,. 자꾸 쓰러져요 반품하기 귀챦아서 그냥 쓰는데 불량 같아요 디자인은 좋아요,"[(., SF), (자꾸, MAG), (쓰러지, VV), (어요, EFN), (반,...","[자꾸, 반, 품, 하기, 귀챦아, 그냥, 불량, 같, 디자인, 좋]"


In [ ]:
# 긍정 3, 4, 5로 세팅

In [27]:
df_train = coupang_crawling('조명', 0, 5)
df_train.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
10583,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,옐로우,2016.01.20,김*순,.,"[(., SF)]",[]
10584,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,실버,2016.01.19,이*선,.,"[(., SF)]",[]
10585,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.17,김*훈,.,"[(., SF)]",[]
10586,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2016.01.05,김*연,.,"[(., SF)]",[]
10587,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.02,김*숙,.,"[(., SF)]",[]


In [28]:
X_train = Vectorizer(df_train, 'pos_filtered', df_train)
y_train = df_train['rating_filtered']

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.82      0.36      0.50       521
          1       0.97      1.00      0.98     10067

avg / total       0.96      0.96      0.96     10588



In [30]:
df_test = coupang_crawling('조명', 6, 10)
df_test.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
6948,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,옐로우,2016.01.20,김*순,.,"[(., SF)]",[]
6949,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,실버,2016.01.19,이*선,.,"[(., SF)]",[]
6950,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.17,김*훈,.,"[(., SF)]",[]
6951,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2016.01.05,김*연,.,"[(., SF)]",[]
6952,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2016.01.02,김*숙,.,"[(., SF)]",[]


In [31]:
X_test = Vectorizer(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [32]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.78      0.29      0.42       326
          1       0.97      1.00      0.98      6627

avg / total       0.96      0.96      0.95      6953



In [33]:
idx = []
for i in range(len(df_test)):
    if len(df_test['pos_filtered'][i]) == 0:
        idx.append(i)

In [34]:
df_test_ = df_test.drop(index=idx).reset_index()

In [35]:
X_test_ = Vectorizer(df_train, 'pos_filtered', df_test_)
y_test_ = df_test_['rating_filtered']

In [36]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred_ = model.predict(X_test_)
print(classification_report(y_test_, y_test_pred_))

             precision    recall  f1-score   support

          0       0.78      0.55      0.65       170
          1       0.98      0.99      0.99      3547

avg / total       0.97      0.97      0.97      3717



In [41]:
df_test.loc[idx]

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
228,5,1,36234235,윤우네무드등 하와이안 무드등,플라밍고(PINK),2018.08.15,윤*영,.,"[(., SF)]",[]
229,3,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.08.15,김*민,.,"[(., SF)]",[]
235,2,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.07.31,김*자,.,"[(., SF)]",[]
247,5,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.07.10,심*희,.,"[(., SF)]",[]
250,5,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.07.03,이*희,.,"[(., SF)]",[]
264,2,0,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.06.19,박*홍,.,"[(., SF)]",[]
288,5,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.05.08,정*현,.,"[(., SF)]",[]
295,5,1,36234235,윤우네무드등 하와이안 무드등,파인애플(YELLOW),2018.05.01,송*,.,"[(., SF)]",[]
297,3,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.04.19,민*화,.,"[(., SF)]",[]
299,5,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2018.04.14,이*영,.,"[(., SF)]",[]


In [ ]:
kkma-2.0.jar

In [28]:
import requests
from scrapy.http import TextResponse
import json
search_url = "http://www.coupang.com/np/search?component=&q=조명"
rep = requests.get(search_url)
response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

# 상품마다의 링크를 불러오기에 앞서 아이템 Top 10 리스트 불러오기
products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][:5]

df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

review_urls = {}

for product in products:

    # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
    try:
#         review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
        review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
    except Exception as e:
        print('No reviews')

    def make_url(product, review_count):
        page_count = (review_count//100)+1
        review_urls = []
        for page in range(1, page_count+1):
            review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
            # size는 100이 최대로 설정되어 있음
        return review_urls


    # Top 10 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
    review_urls[product] = make_url(product, review_count)
    review_url = make_url(product, review_count) # 리스트로 반환됨
#     print(review_url)

    for url in review_url:

        review_url = url
        review_rep = requests.get(review_url)
        review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

        # Top 10 상품마다의 상품평 개수
        review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

        # Top 10 상품마다 한 page의 상품평 개수
        review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

        if review_url_cnt < 100:          
            for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

        else:
            for i in range(1, 101):
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

In [29]:
len(df)

5873

In [30]:
df.tail()

,rating,item_nbr,item,option,date,name,re_title,review
5868,3,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.26,박*화,[.],[]
5869,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.26,차*지,[.],[]
5870,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.20,민또,[.],[]
5871,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.16,정*현,[.],[]
5872,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.11.16,강*희,[.],[]


In [78]:
def re_title_filter(a):
    for i in range(len(a[0].split('\n'))):
        if len(a[0].split('\n')[i].strip()) != 0:
            return a[0].split('\n')[i].strip()
        else:
            return '.'

def review_filter(a):
    s = ''
    for i in range(len(a)):
        for ii in range(len(a[i].split('\n'))):
            if a[i].split('\n')[ii].strip() != 0:
                s += a[i].split('\n')[ii].strip() + ' '
    return s

def kkma_pos(a):
    from konlpy.tag import Kkma
    kkma = Kkma()
    return kkma.pos(a)

def kkma_pos_filter(a):
    """
    df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
    """
    ls = []
    for i in range(len(a)):
        #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
        if a[i][1]=='NNG' or a[i][1]=='NNP' or a[i][1]=='VA' or a[i][1]=='UN' or a[i][1]=='XR' or a[i][1]=='MAG':
            ls.append(a[i][0])
    return ls

def rating_filter(a):
    if a == '4' or a == '5':
        return 1
        # 긍정
    else:
        return 0
        # 부정

In [79]:
df_ = df[253:255]
df_['re_title_filtered'] = df_['re_title'].apply(re_title_filter)
df_['review_filtered'] = df_['review'].apply(review_filter)
df_['full_review'] = df_['re_title_filtered']+df_['review_filtered']
df_['pos'] = df_['full_review'].apply(kkma_pos)
df_['pos_filtered'] = df_['pos'].apply(kkma_pos_filter)
df_['rating_filtered'] = df_['rating'].apply(rating_filter)

df_.tail(3)
# df_fin = df_[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
# df_fin.tail(3)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,rating,item_nbr,item,option,date,name,re_title,review,re_title_filtered,review_filtered,full_review,pos,pos_filtered,rating_filtered
253,5,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.04.11,박*기,[\n \n \n ...,[\n \n ...,.,너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립되...,. 너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립...,"[(., SF), (너무, MAG), (이쁘, VA), (어요, EFN), (ㅠㅠㅠ...","[너무, 이쁘, 진짜, 길, 생각, 가격, 저렴, 혼자, 척척, 잘, 조립, 그냥,...",1
254,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.02.09,박*서,[\n \n 배송상태 ...,[\n \n ...,.,구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 고...,. 구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 ...,"[(., SF), (구멍, NNG), (이, VCP), (란, ETD), (구멍, ...","[구멍, 구멍, 전부, 스트로, 폼, 잔뜩, 제거, 면봉, 이쑤시개, 고생, 평, ...",0


In [61]:
df_

,rating,item_nbr,item,option,date,name,re_title,review,re_title_filtered,review_filtered,full_review
253,5,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.04.11,박*기,[\n \n \n ...,[\n \n ...,None,너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립되...,NaN
254,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.02.09,박*서,[\n \n 배송상태 ...,[\n \n ...,"배송상태 불량 200%, 제품상태 -200%",구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 고...,"배송상태 불량 200%, 제품상태 -200% 구멍이란 구멍엔 전부 스트로폼이 잔뜩..."


In [62]:
df_['re_title']

253    [\n                \n                     \n  ...
254    [\n                \n                    배송상태 ...
Name: re_title, dtype: object

In [77]:
df_['re_title'][254][0].split('\n')

for i in range(len(df_['re_title'][254][0].split('\n'))):
    if len(df_['re_title'][254][0].split('\n')[i].strip()) != 0:
        print(df_['re_title'][254][0].split('\n')[i].strip(), type(df_['re_title'][254][0].split('\n')[i].strip()))
    else:
        print('none')

none
none
배송상태 불량 200%, 제품상태 -200% <class 'str'>
none
none


In [ ]:
def re_title_filter(a):
    for i in range(len(a[0].split('\n'))):
        if len(a[0].split('\n')[i].strip()) != 0:
            return a[0].split('\n')[i].strip()